<a href="https://colab.research.google.com/github/rohithmsr/AI-practice/blob/main/Opik/0_intro_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Logging Traces with the Open AI Integration

In this exercise, you'll log some basic traces with Opik. You can use OpenAI or open source models via LiteLLM. You can find [the full documentation for Opik's integration with OpenAI here](https://www.comet.com/docs/opik/tracing/integrations/openai). You can find [the full documentation for Opik's integration with LiteLLM here](https://www.comet.com/docs/opik/cookbook/litellm).

# Imports & Configuration

In [ ]:
%pip install opik openai --quiet

In [ ]:
import opik
import os
import getpass

# Define project name to enable tracing
os.environ["OPIK_PROJECT_NAME"] = "logging_traces_demo"

In [ ]:
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Tracking OpenAI Calls

In [ ]:
from opik.integrations.openai import track_openai
from openai import OpenAI

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
openai_client = track_openai(openai_client)

In [ ]:
prompt="Hello, world!"

response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role":"user", "content":prompt}
    ],
    temperature=0.7,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(response.choices[0].message.content)

# Use Open Source Models With LiteLLM
Opik also integrates with LiteLLM, which allows you to use free open-source models and supports LLM APIs from all of the major providers (Bedrock, Huggingface, VertexAI, TogetherAI, Azure, OpenAI, Groq, etc.) using the OpenAI format. [See here for a full list of LLM providers supported by LiteLLM as well as how to call them.](https://docs.litellm.ai/docs/providers)

In the following example we'll use Meta's `Llama-3.1-8B-Instruct` model hosted on the Hugging Face hub.

**If you have already run the OpenAI code above, you will need to restart your kernel before running the following code**

In [ ]:
%pip install opik litellm --quiet

In [ ]:
import opik
import os
import getpass

os.environ["OPIK_PROJECT_NAME"] = "logging-traces-litellm"

if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

In [ ]:
from litellm.integrations.opik.opik import OpikLogger
from opik.opik_context import get_current_span_data
from opik import track
import litellm

opik_logger = OpikLogger()
# In order to log LiteLLM traces to Opik, you will need to set the Opik callback
litellm.callbacks = [opik_logger]

In [ ]:
import os
import getpass

if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")

In [ ]:
messages = [{ "content": "There's a llama in my garden 😱 What should I do?","role": "user"}]

response = litellm.completion(
    model="huggingface/meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=messages
)

print(response.choices[0].message.content)